In [0]:
external_azure_path = 'abfss://container20260122@datalake20260121.dfs.core.windows.net/'

dbutils.fs.ls(f'{external_azure_path}/01-bronze')

In [0]:
# Creating tmp tables to load product data into silver layer
allowed_tables = frozenset({'products/', 'categories/', 'brands/', 'stocks/'})

for delta_table in dbutils.fs.ls(f'{external_azure_path}/01-bronze'):
    if delta_table.name in allowed_tables:
        (
            spark.read
                .format("delta")
                .load(delta_table.path)
                .createOrReplaceTempView(f'tmp_bronze_{delta_table.name.rstrip('/')}')
        )

In [0]:
%sql
DESCRIBE tmp_bronze_products 

In [0]:
%py
df_silver_products = spark.sql('''
    -- Creating SQL query to aggregate product data
    WITH cte_stocks AS (
      SELECT 
          product_id
        , SUM(quantity) AS total_stock
      FROM 
        tmp_bronze_stocks
      GROUP BY 
        product_id
    )
    
    SELECT
        pd.product_id
      , pd.product_name
      , bd.brand_id
      , ct.category_id
      , pd.model_year
      , pd.list_price
      , st.total_stock
    FROM
      tmp_bronze_products             AS pd
      LEFT JOIN tmp_bronze_categories AS ct ON pd.category_id = ct.category_id
      LEFT JOIN tmp_bronze_brands     AS bd ON pd.brand_id    = bd.brand_id
      LEFT JOIN cte_stocks            AS st ON pd.product_id  = st.product_id
''')

display(df_silver_products)

In [0]:
# Writing on silver azure external storage in delta format
(df_silver_products.write
    .mode('overwrite')
    .format('delta')
    .option('mergeSchema', 'true')
    .save(f'{external_azure_path}/02-silver/products')
)

In [0]:
%sql
-- Creating a new schema to store silver tables
CREATE SCHEMA IF NOT EXISTS bike_store.silver_schema

In [0]:
%sql
DROP TABLE IF EXISTS bike_store.silver_schema.products

In [0]:
%sql
-- Creating databricks delta table
CREATE TABLE IF NOT EXISTS bike_store.silver_schema.products
LOCATION 'abfss://container20260122@datalake20260121.dfs.core.windows.net/02-silver/products'